## Εργασία 3 ##

Καλωσήρθατε στην τρίτη σας εργασία. Η εργασία αυτή έχει σκοπό να σας βοηθήσει να εμπεδώσετε τα σύνολα μοντέλων.

Στην εργασία αυτή θα πρέπει να συμπληρώσετε κώδικα Python 3 στα σημεία που αναφέρουν # YOUR CODE HERE. Μην τροποποιείτε τον κώδικα που βρίσκεται εκτός αυτών των περιοχών.

Πρωτού παραδόσετε την εργασία σας σιγουρευτείτε ότι ο κώδικας σε όλα τα κελιά τρέχει σωστά. Για το σκοπό αυτό επιλέξτε από το μενού Χρόνος εκτέλεσης (runtime) -> Επανεκίνηση περιόδου λειτουργίας και εκτέλεση όλων.

Συμπληρώστε το όνομα (NAME) και το AEM σας παρακάτω:

In [1]:
NAME = "Christos Oikonomou"
AEM = "10268"

**1** Διαβάστε το διαθέσιμο από την sklearn σύνολο δεδομένων breast cancer, χωρίστε το σε δεδομένα εκπαίδευσης (X_train, y_train) και ελέγχου (X_test, y_test) σε ποσοστό 70%/30% αντίστοιχα με τη συνάρτηση train_test_split (τιμή για random_state βάλτε 0). Το σύνολο αφορά τη διάγνωση καρκίνου του μαστού με βάση μεταβλητές που υπολογίζονται από μια ψηφιοποιημένη εικόνα δείγματος μάζας μαστού που λήφθηκε μέσω αναρρόφησης λεπτής βελόνας (FNA). (2 μονάδες)

In [2]:
# YOUR CODE HERE
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_breast_cancer

breast_cancer = load_breast_cancer()
X = breast_cancer.data
y = breast_cancer.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, train_size = 0.7, random_state = 0)

In [3]:
"""Τεστ ορθής ανάγνωσης και διαχωρισμού του συνόλου δεδομένων"""
assert round(X_train[0][8], 5) == 0.1779
assert round(X_test[0][8], 5) == 0.2116

**2** Υλοποιήστε μια ντετερμινιστική εκδοχή της μεθόδου των τυχαίων υποχώρων, η οποία χτίζει τόσα μοντέλα όσες και οι μεταβλητές εισόδου, κάθε ένα από τα οποία αγνοεί και μία διαφορετική μεταβλητή εισόδου. Π.χ. το πρώτο μοντέλο αγνοεί την πρώτη, το δεύτερο αγνοεί τη δεύτερη κτλ. Χρησιμοποιήστε τη συνάρτηση clone από το sklearn.base για να δημιουργήστε αντίγραφο του βασικού μοντέλου σε κάθε επανάληψη. (4 μονάδες)

In [4]:
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.base import clone

class RandomSubspaceDet:
    def __init__(self, estimator=DecisionTreeClassifier()):
        # YOUR CODE HERE
        self.estimator = estimator
        self.models = []
        self.n_features_ = 0

    def fit(self, X_train, y_train):
        # YOUR CODE HERE
        self.n_features_ = X_train.shape[1]
        self.models = []

        for i in range(self.n_features_):
          # Create a copy of the estimator
          model = clone(self.estimator)
          # Exclude the i-th feature
          X_train_subset = np.delete(X_train, i, axis=1)
          # Fit the model
          model.fit(X_train_subset, y_train)
          self.models.append(model)

    def predict(self, X):
        # YOUR CODE HERE

        # Collect predictions from all models
        all_predictions = []

        for i, model in enumerate(self.models):
            # Exclude the i-th feature
            X_subset = np.delete(X, i, axis=1)
            # Predict using the model
            predictions = model.predict(X_subset)
            all_predictions.append(predictions)

        # Transpose to have predictions for each sample in rows
        all_predictions = np.array(all_predictions).T

        # Get the majority vote for each sample
        majority_vote_predictions = []
        for preds in all_predictions:
            unique, counts = np.unique(preds, return_counts=True)
            majority_vote = unique[np.argmax(counts)]
            majority_vote_predictions.append(majority_vote)

        return np.array(majority_vote_predictions)

In [5]:
"""Τεστ ορθής υλοποίησης RandomSubspaceDet"""
from sklearn.metrics import accuracy_score

rs = RandomSubspaceDet(estimator=DecisionTreeClassifier(random_state=1))
rs.fit(X_train, y_train)
assert round(accuracy_score(rs.predict(X_test), y_test), 4) == 0.9006

**3** Υλοποιήστε τη μέθοδο AdaBoost όπως παρουσιάστηκε στο μάθημα. Χρησιμοποιήστε τη συνάρτηση clone από το sklearn.base για να δημιουργήστε αντίγραφο του βασικού μοντέλου σε κάθε επανάληψη. Χρησιμοποιήστε την παράμετρο sample_weight της fit του βασικού μοντέλου για να ορίσετε τα βάρη των παραδειγμάτων εκπαίδευσης. (4 μονάδες)

In [6]:
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.base import clone

class AdaBoost:
    def __init__(self, n_estimators=20, estimator=DecisionTreeClassifier(max_depth=1)):
        # YOUR CODE HERE
        self.n_estimators = n_estimators
        self.base_estimator = estimator
        self.estimators_ = []
        self.alphas_ = []

    def fit(self, X_train, y_train):
        # YOUR CODE HERE
        n_samples = X_train.shape[0]
        # Initialize weights
        w = np.ones(n_samples) / n_samples

        # Convert y_train to -1, 1
        y_train_converted = np.where(y_train == 0, -1, 1)

        for _ in range(self.n_estimators):
            # Create a new estimator and fit it to the data with the current weights
            estimator = clone(self.base_estimator)
            estimator.fit(X_train, y_train_converted, sample_weight=w)
            y_pred = estimator.predict(X_train)

            # Calculate the error
            incorrect = (y_pred != y_train_converted)
            error = np.dot(w, incorrect) / np.sum(w)

            # Calculate the estimator weight
            alpha = 0.5 * np.log((1 - error) / (error + 1e-10))  # Adding a small value to avoid division by zero

            # Update the weights
            w *= np.exp(-alpha * y_train_converted * y_pred)
            w /= np.sum(w)  # Normalize weights

            # Save the current estimator and its weight
            self.estimators_.append(estimator)
            self.alphas_.append(alpha)

    def predict(self, X):
        # YOUR CODE HERE
        # Aggregate predictions from each estimator
        estimator_preds = np.array([alpha * est.predict(X) for est, alpha in zip(self.estimators_, self.alphas_)])
        # Return the sign of the weighted sum of predictions
        y_pred = np.sign(np.sum(estimator_preds, axis=0))
        # Convert -1, 1 back to 0, 1
        y_pred = np.where(y_pred == -1, 0, 1)
        return y_pred

In [7]:
"""Τεστ ορθής υλοποίησης AdaBoost"""
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score

ab = AdaBoost(n_estimators=20, estimator=DecisionTreeClassifier(max_depth=1, random_state=1))
ab.fit(X_train, y_train)
assert round(accuracy_score(ab.predict(X_test), y_test), 4) == 0.9591


In [8]:
# Ίδιο αποτέλεσμα και με τη κλάση της sklearn
ab = AdaBoostClassifier(n_estimators=20, algorithm="SAMME", estimator=DecisionTreeClassifier(max_depth=1, random_state=1))
ab.fit(X_train, y_train)
assert round(accuracy_score(ab.predict(X_test), y_test), 4) == 0.9591